### CephFS 

Wir verwenden [Host Storage Cluster](https://rook.io/docs/rook/v1.11/CRDs/Cluster/host-cluster/)

Überprüfen, ob die CephFS-CSI-Plugins korrekt laufen:

In [ ]:
%%bash
kubectl -n rook-ceph get pods -l app=csi-cephfsplugin

Zuerst brauchen wie ein Filesystem

In [ ]:
%%bash
kubectl apply -f https://raw.githubusercontent.com/rook/rook/1cf0a83ead305f0dab510827269bbde495765bc3/deploy/examples/filesystem-test.yaml

In [ ]:
%%bash
kubectl apply -f - <<EOF
apiVersion: ceph.rook.io/v1
kind: CephFilesystem
metadata:
  name: myfs

spec:
  metadataPool:
    replicated:
      size: 3
  dataPools:
    - replicated:
        size: 3
  preserveFilesystemOnDelete: true
  metadataServer:
    activeCount: 1
    activeStandby: true
EOF

Prüfen, ob CephFS korrekt eingerichtet ist
Überprüfe, ob CephFS bereits läuft:

In [ ]:
%%bash
kubectl -n rook-ceph get CephCluster,cephfilesystems,CephFilesystemSubVolumeGroup
kubectl -n rook-ceph describe CephCluster

*Evtl. könnten die Zugriffsberechtigungen RBAC nicht vorhanden sein, deshalb setzen wir diese*

In [ ]:
%%bash
kubectl apply -f - <<EOF
apiVersion: rbac.authorization.k8s.io/v1
kind: ClusterRole
metadata:
  name: ceph-csi-provisioner
rules:
  - apiGroups: [""]
    resources: ["persistentvolumes"]
    verbs: ["get", "list", "watch", "create", "delete", "patch", "update"]
  - apiGroups: ["storage.k8s.io"]
    resources: ["storageclasses"]
    verbs: ["get", "list", "watch"]
  - apiGroups: ["storage.k8s.io"]
    resources: ["volumeattachments"]
    verbs: ["get", "list", "watch", "patch", "update"]
---
apiVersion: rbac.authorization.k8s.io/v1
kind: ClusterRoleBinding
metadata:
  name: ceph-csi-provisioner-binding
roleRef:
  apiGroup: rbac.authorization.k8s.io
  kind: ClusterRole
  name: ceph-csi-provisioner
subjects:
  - kind: ServiceAccount
    name: rook-csi-cephfs-provisioner
    namespace: rook-ceph
EOF
# braucht Restart
kubectl -n rook-ceph delete pod -l app=csi-cephfsplugin-provisioner

### StorageClass für Rook-Ceph erstellen.

Mit dieser werden keine PV erstellt.

Alternative microk8s-hostpath vom Addon hostpath-storage verwenden

In [ ]:
%%bash
kubectl apply -f - <<EOF
apiVersion: storage.k8s.io/v1
kind: StorageClass
metadata:
  name: rook-cephfs 
provisioner: rook-ceph.cephfs.csi.ceph.com
parameters:
  clusterID: rook-ceph
  fsName: myfs
  pool: myfs-data0
  csi.storage.k8s.io/provisioner-secret-name: rook-csi-cephfs-provisioner
  csi.storage.k8s.io/provisioner-secret-namespace: rook-ceph
  csi.storage.k8s.io/node-stage-secret-name: rook-csi-cephfs-node
  csi.storage.k8s.io/node-stage-secret-namespace: rook-ceph
  csi.storage.k8s.io/fstype: ext4
reclaimPolicy: Delete
EOF

### PersistentVolume und Persistent Volume Claim (PVC) für Ceph RBD erstellen

**Hinweis** PersistentVolume sollte eigentlich automatisch erstellt werden.

### Hole die benötigten Informationen für das PV

Die Ceph-Monitor-Adressen

In [ ]:
%%bash
kubectl -n rook-ceph get cm rook-ceph-mon-endpoints -o jsonpath='{.data.data}'

In [ ]:
%%bash
kubectl apply -f - <<EOF
apiVersion: v1
kind: PersistentVolume
metadata:
  name: cephfs-pv
spec:
  capacity:
    storage: 5Gi  # Größe des Volumes anpassen
  accessModes:
    - ReadWriteMany  # CephFS unterstützt RWX
  persistentVolumeReclaimPolicy: Retain
  storageClassName: rook-cephfs   # Muss mit deiner StorageClass übereinstimmen
  csi:
    driver: rook-ceph.cephfs.csi.ceph.com
    volumeHandle: myfs  # Name des CephFS-Dateisystems
    volumeAttributes:
      clusterID: rook-ceph
      fsName: myfs
      pool: myfs-data0
      provisionVolume: "true" 
      monitors: "10.152.183.173:6789"
    nodeStageSecretRef:
      name: rook-csi-cephfs-node
      namespace: rook-ceph
EOF


In [ ]:
%%bash
kubectl apply -f - <<EOF
apiVersion: v1
kind: PersistentVolumeClaim
metadata:
  name: ceph-pvc

spec:
  accessModes:
    - ReadWriteMany
  resources:
    requests:
      storage: 5Gi
  storageClassName: rook-cephfs
  volumeName: cephfs-pv  # Name des vorher erstellten PVs
EOF

Wir kontrollieren ob alles sauber erstellt wurde

In [ ]:
%%bash
kubectl describe pvc ceph-pvc

Pod mit Ceph-Speicher starten

In [ ]:
%%bash
kubectl apply -f - <<EOF
apiVersion: v1
kind: Pod
metadata:
  name: ceph-test-pod

spec:
  containers:
  - name: busybox
    image: busybox
    command: ["/bin/sh", "-c", "while true; do echo 'Ceph läuft!' >> /mnt/test/data.txt; sleep 10; done"]
    volumeMounts:
    - mountPath: "/mnt/test"
      name: ceph-storage
  volumes:
  - name: ceph-storage
    persistentVolumeClaim:
      claimName: ceph-pvc
EOF

In [ ]:
%%bash
kubectl describe pod ceph-test-pod

---

### Aufräumen

In [ ]:
%%bash
kubectl delete pod ceph-test-pod 
kubectl delete pvc ceph-pvc
kubectl delete pv cephfs-pv 
kubectl delete sc rook-cephfs


### Tests

In [ ]:
%%bash
kubectl -n rook-ceph exec deploy/rook-ceph-tools -- ceph status
